**GEO6361, semaine 11** : Utilisation d'API

Voyons comment se connecter à des bases de données distantes en utilisant des APIs.

In [ ]:
%%capture
!pip install mapclassify

## **1. Automatiser l'extraction de données par API**

### **1.1 Importons les modules requis pour cette section**

In [ ]:
import pandas as pd # On importe pandas, on lui attribut l'alias pd
import geopandas as gpd # On import GeoPandas et on lui attribut l'alias gpd
import matplotlib.pyplot as plt # On importe Matplotlib pour afficher figures et cartes
import seaborn as sns # On importe SeaBorn qui ajoute des fonctionnalités à Matplotlib
import mapclassify

### **1.2 API de l'USGS**

L'institut d'études géologiques des États-Unis (USGS) propose des flux de données issus d'un réseau international de sismographes.

L'API est disponible à cette adresse : https://earthquake.usgs.gov/earthquakes/feed/. Les données peuvent être récupérées sous différents formats (csv, geojson, kml, etc.).

Connectons-nous à cette API pour récupérer un CSV et effectuer des analyses simples. Commençons par récupérer le fichier contenant tous les événements sismiques des 30 derniers jours (all_month.csv sur la page https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php)

In [ ]:
# Charger le contenu dans une DataFrame Pandas
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')

In [ ]:
# Nombre d'enregistrements récupérés grâce à l'API


In [ ]:
# Afficher la liste des colonnes
 # explication des colonnes : https://earthquake.usgs.gov/data/comcat/

In [ ]:
# On convertit le type des données pour refléter leur contenu
df['time'] = pd.to_datetime(df['time']) # Pour interpréter le contenu de la colonne time comme des dates

In [ ]:
print(f"La date de l'évènement le plus ancien dans les données : {df['time'].min().day}/{df['time'].min().month}/{df['time'].min().year}")
print(f"La date de l'évènement le plus récent dans les données : {df['time'].max().day}/{df['time'].max().month}/{df['time'].max().year}")

Nous voyons que les données ont une colonne "type", quels sont ces types ?

In [ ]:
# Quels sont les différents types de séismes enregistrés ?
df['type'].unique()

Visualisons la distribution de la magnitude pour chacun de ces types :

In [ ]:
sns.set_style("whitegrid")
fig = sns.displot(df, # Voir la doc : https://seaborn.pydata.org/generated/seaborn.displot.html
                  x="mag", # la colonne à visualiser
                  hue="type", # la catégorie à partir de laquelle nous voulons séparer l'histogramme: ici type
                  height=5, # la hauteur de la figure
                  aspect=1.5, # le rapport hauteur/largeur
                  fill=True, # remplir l'histogramme d'une couleur (avec transparence)
                  kind="hist", # type de représentation (on peut aussi passer "kde" pour calculer la densité)
                  binwidth=.1, # largeur des classes
                # common_norm=False # Utile pour la visualisation par densité
                  )
fig.set(xlim=(0, 8)) # limites de l'axe des abscisses, pour limiter l'emprise des données
fig.set(title='Distribution des évènements sismiques', xlabel='Magnitude', ylabel='Fréquence') # Pour modifier les titres et les noms des axes
fig._legend.set_title("Type d'évènement") # Titre de la légende

Les données ont une dimension spatiale (les coordonnées géographiques dans les colonnes "longitude" et "latitude"), créons une GeoDataFrame à partir de la DataFrame :

In [ ]:
# On convertit la DataFrame en GeoDataFrame
gdf = #### (
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude) # df.longitude est équivalent à df['latitude']
)

# On renseigne son système de coordonnées
gdf.crs = 'EPSG:4326'

Visualisons les données spatiales :

In [ ]:
# Un rapide coup d'oeil aux données avec la méthode plot()


In [ ]:
# On charge les données d'un fond de carte venant de https://www.naturalearthdata.com
fond = gpd.read_file('/content/fond_natural_earth.geojson')

In [ ]:
# On crée un cadre pour la carte
fig, ax = plt.subplots(figsize=(15,15))

# On affiche le fond de carte dans ce cadre
fond.plot(ax=ax, alpha=0.4, color='grey')

# On affiche les données de la GeoDataFrame par dessus le fond de carte
gdf.plot(column='type', markersize=1, cmap='prism', ax=ax, legend=True) # jeu de couleurs disponibles: https://matplotlib.org/stable/users/explain/colors/colormaps.html

# On ajoute un titre
plt.title('Séismes (30 derniers jours).')

# On peut sauvegarder l'image
from datetime import datetime
date = datetime.today().strftime('%d-%m-%Y')
plt.savefig(f'/content/Séismes_30_derniers_jours_{date}.png', bbox_inches='tight', dpi=300)

Utilisons un système de cartographie interactive avec le module "Folium" pour afficher une carte de chaleur (heatmap) :

In [ ]:
import folium
from folium import plugins

# On crée une carte dynamique Folium (vide)
carte_seismes = folium.Map(location=[0,0], tiles='Cartodb Positron', width="100%", height="100%", zoom_start=2)

#print(gdf.geometry.head())
#print(gdf.geometry[0])
#print(gdf.geometry[0].xy)

# On extrait les données sous la forme latitude-longitude depuis la colonne gemetry (format WKT)
heat_data = []
for point in gdf.geometry:
    heat_data.append([point.xy[1][0], point.xy[0][0]])

# On ajoute une carte de chaleur (heatmap) à la carte dynamique Folium
plugins.HeatMap(heat_data).add_to(carte_seismes) # On doit fournir une liste de données de la forme latitude-longtitude

# On affiche la carte
carte_seismes

### **1.3 API d'Inaturalist**

Inaturalist possède une API (https://pyinaturalist.readthedocs.io/en/stable/) permettant de soumettre des observations de manière automatique, ainsi que d'interroger la base de données. Essayons de télécharger des données qui pourront être réutilisées dans des analyses Python.

In [ ]:
# Installer Pyinaturalist
%%capture
!pip install pyinaturalist

In [ ]:
# Importons les fonctions Pyinaturalist dont nous aurons besoin
from pyinaturalist import get_observations, get_places_autocomplete

Nous pouvons effectuer des recherches par lieux. Les lieux sont codifiés dans la base de données d'Inaturalist. Détectons ici quel est le code pour le Québec


In [ ]:
# Faisons une requête sur la base de données pour le terme générique "quebec"
response = get_places_autocomplete(q='quebec')

# Affichons le contenu de la réponse
for i in response['results']:
    print(i['id'], i['name'])

In [ ]:
# Interrogeons la base de données d'Inaturalist :
response = get_observations(
    taxon_name='Cirsium vulgare', # indiquer le taxon
    #taxon_name='Ulmus pumila',
    #taxon_name='Odocoileus virginianus',
    d1='2023', # Date de l'observation la plus ancienne que l'on souhaite récupérer
    d2='2024', # Date de l'observation la plus récente
    geo=True, # Indiquer que nous souhaitons récupérer les données possédant une dimension spatiale
    geoprivacy='open',
    place_id=13336, # Identifiant du lieu sur lequel nous souhaitons faire la recherche
    page='all' # Récupérer tous les résultats
)

In [ ]:
# Affichons le nombre de résultats contenus dans la réponse à notre requête
print(f"{len(response['results'])} observations ont été récupérées sur le site d'iNaturalist.")
print(f"{response['total_results']} observations ont été récupérées sur le site d'iNaturalist.")

# Stockons le contenu de la réponse dans une DataFrame
df = pd.json_normalize(response['results']) # Voir https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html

In [ ]:
df['location']

In [ ]:
# Créons de nouvelles colonnes pour stocker les coordonnées géographiques issues de la requête :
df['lat'] = df['location'].apply(lambda x: x[0])
df['lng'] = df['location'].apply(lambda x: x[1])

In [ ]:
# La dataframe contient beaucoup d'informations, simplifions-là en ne retenant que les informations utiles pour nous :
df = df[['observed_on_details.date', 'quality_grade', 'taxon_geoprivacy', 'lat', 'lng']]

In [ ]:
# Comme les données sont géoréférencées, nous pouvons transformer la DatFrame en GeoDataFrame :
gdf = ###(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))

In [ ]:
# Affichons les données et faisons ressortir leur niveau de qualité
gdf.plot(
    column='quality_grade',
    legend=True,
    figsize=(20,20))

In [ ]:
# On crée une carte dynamique Folium (vide) centrée sur
carte_inaturalist = folium.Map(location=[51.265736539784356, -72.8824016746752], tiles='Cartodb Positron', width="90%", height="90%", zoom_start=5)

# On extrait les données sous la forme latitude-longitude depuis la colonne gemetry (format WKT)
heat_data = []
for point in gdf.geometry:
    heat_data.append([point.xy[1][0], point.xy[0][0]])

# On ajoute une carte de chaleur (heatmap) à la carte dynamique Folium
plugins.HeatMap(heat_data).add_to(carte_inaturalist)

# On affiche la carte
carte_inaturalist

## **2. Un peu d'analyse des réseaux**

### **2.1 API d'Open Street Map**

Utilisons ici **OSMNX** (https://github.com/gboeing/osmnx), un module conçu pour extraire de nombreuses informations d'Open Street Map sous forme :
* de réseaux (voirie, réseaux techniques)
* de polygones (ex : emprise du bâti)
* ou bien des méta-données sur ces objets (restaurant, résidentiel, etc.).

Voyons ici un court exemple d'**analyse de réseau de voirie**...

Pour commencer, installons OSMNX :

In [ ]:
%%capture
!pip install osmnx

In [ ]:
# Importons les modules utiles à notre démonstration
import osmnx as ox
ox.settings.log_console=True
ox.settings.use_cache=True

# Pour l'analyse de réseaux
import networkx as nx

# Classiques
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

Nous pouvons maintenant importer les données d'un réseau routier d'intérêt :

In [ ]:
# À partir d'un nom de lieu
#ville = "Montréal, Quebec"
#reseau = ox.graph_from_place(ville, network_type='drive', simplify=True)

# À partir d'un point géographique
lieu = 45.5230, -73.6195 # Ici, campus MIL
dist = 1500 # distance autour du point (buffer) en mètres
reseau = ox.graph_from_point(lieu, dist=dist, network_type='drive', simplify=True)

print(f"Le réseau de voirie de cette zone compte {len(reseau.nodes)} intersections et {len(reseau.edges)} segments.")

Et l'afficher de manière à représenter les nœuds du réseau et les liens entre eux :

In [ ]:
# Visualisons rapidement le réseau obtenu
fig, ax = ox.plot_graph(reseau, node_size=20, bgcolor='w', node_color='w', node_edgecolor='k', node_zorder=1, figsize=(10,10))

Créons une carte interactive :

In [ ]:
# Grâce à la méthode explore() de GeoPandas
ox.graph_to_gdfs(reseau, nodes=False).explore()

Calculons quelques mesures de centralité (cf. des mesures du niveau auquel les nœuds du réseaux sont "centraux", ou contribuent à la connectivité générale du réseau). Plus particulièrement, nous mesurerons la **centralité "betweeness"** (la fréquence à laquelle chacun des nœuds se trouve sur le plus court chemin entre chaque paire de nœuds dans le réseau). **D'après Goeff Being (https://github.com/gboeing/osmnx)**. Pour plus d'exemples, voir la page: https://github.com/gboeing/osmnx-examples/tree/main/notebooks

In [ ]:
reseau_oriente = nx.DiGraph(reseau)
mesure_bet_centrality = nx.betweenness_centrality(reseau_oriente)

In [ ]:
# Créeons une dataframe
df = ###(data=pd.Series(mesure_bet_centrality).sort_values(), columns=['centrality'])

# Configurons l'affichage
df['colors'] = ox.plot.get_colors(n=len(df), cmap='inferno', start=0.2)
df = df.reindex(reseau.nodes())

node_centrality = df['colors'].tolist()

# Affichons le réseau sur une figure
fig, ax = ox.plot_graph(reseau, bgcolor='k', node_size=30, node_color=node_centrality, node_edgecolor='none', edge_color='#555555', edge_linewidth=1.5, edge_alpha=1, figsize=(10,10))

# fig.savefig(f'/content/MIL_nodes.png', bbox_inches='tight', dpi=300)

Représentons cette mesure en inversant les nœuds et les arêtes (représentation plus fréquente pour ce genre de problème...)

In [ ]:
edge_centrality = nx.betweenness_centrality(nx.line_graph(reseau_oriente))

In [ ]:
edge_values = []
for edge in reseau.edges():
    edge_values.append(edge_centrality[edge])

cmap = cm.ScalarMappable(norm=colors.Normalize(vmin=min(edge_values)*0.8, vmax=max(edge_values)), cmap=cm.inferno)

edge_colors = []
for cl in edge_values:
    edge_colors.append(cmap.to_rgba(cl))

fig, ax = ox.plot_graph(reseau, bgcolor='k', node_size=0, node_color=node_centrality, node_edgecolor='none', edge_color=edge_colors, edge_linewidth=3.5, edge_alpha=1, figsize=(10,10))

# fig.savefig(f'/content/MIL_edges.png', bbox_inches='tight', dpi=300)

Nous pouvons également **convertir le réseau en GeoDataFrame** :

In [ ]:
# On récupère dans deux dataframes séparées, respectivement l'ensemble des intersections (nodes) et l'ensemble des rues (edges)
intersections, rues = ox.graph_to_gdfs(reseau)

Et visualisiser **toutes sortes de statistiques** :

In [ ]:
# Analysons le type des rues
types_rues = pd.DataFrame(rues["highway"].apply(pd.Series)[0].value_counts().reset_index())
types_rues.columns = ["type", "nombre"]

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
plt.pie(types_rues['nombre'], labels = types_rues['type'], autopct='%.0f%%')
plt.show()

In [ ]:
# Affichons les rues
rues.plot(figsize=(10,10))

Maintenant, regardons l'empreinte du bâti.

In [ ]:
t = {'building':True}
# t = {'amenity':True, 'landuse':['retail','commercial']}

In [ ]:
# Faisons une requête auprès d'Open Street Map concernant le bâti
bati = ox.features_from_point(lieu, tags=t, dist=1500)
bati.shape

In [ ]:
# Affichons le résultat
ox.plot_footprints(bati, color='w', figsize=(10,10))